In [ ]:
import os
import tempfile
import subprocess
import sys
import json
import random

def load_revised_pages():
    try:
        with open('revised_pages.json', 'r') as f:
            return set(json.load(f))
    except FileNotFoundError:
        return set()

def save_revised_pages(pages):
    with open('revised_pages.json', 'w') as f:
        json.dump(list(pages), f)

def get_random_unrevised_page():
    revised_pages = load_revised_pages()
    all_pages = set(range(1, 605))  # Pages 1-604
    unrevised_pages = all_pages - revised_pages
    
    if not unrevised_pages:
        print("🎉 Congratulations! You have revised all 604 pages of the Quran!")
        return None
    
    return random.choice(list(unrevised_pages))

def install_playwright():
    try:
        subprocess.run([sys.executable, "-m", "pip", "install", "playwright"], check=True)
        subprocess.run([sys.executable, "-m", "playwright", "install", "chromium"], check=True)
        return True
    except:
        return False

def save_page_as_image(page_num):
    ar = requests.get(f"http://api.alquran.cloud/v1/page/{page_num}/quran-uthmani").json()["data"]["ayahs"]
    en = requests.get(f"http://api.alquran.cloud/v1/page/{page_num}/en.sahih").json()["data"]["ayahs"]
    
    # Get surah information for the page
    surahs_on_page = set()
    for ayah in ar:
        surahs_on_page.add(ayah['surah']['englishName'])
    
    surah_titles = " • ".join(sorted(surahs_on_page))
    
    # Arabic + translation
    arabic_translation = []
    for a,e in zip(ar,en):
        arabic_translation.append(f"<div style='margin:10px 0'><div style=\"direction:rtl;text-align:right;font-family:'Amiri','Scheherazade New','Times New Roman',serif;font-size:24px;line-height:2.2\">{a['text']} <span style='color:#d4af37;font-size:.8em;margin-left:10px'>{a['numberInSurah']}</span></div><div style=\"font-family:'Inter',-apple-system,BlinkMacSystemFont,'Segoe UI',Roboto,Arial,sans-serif;font-size:16px;line-height:1.6;color:#eee\">{e['text']}</div></div>")
    
    # Save Arabic + translation
    translation_html = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <meta charset="UTF-8">
        <link href="https://fonts.googleapis.com/css2?family=Amiri:wght@400;700&family=Scheherazade+New:wght@400;700&display=swap" rel="stylesheet">
        <style>
            body {{ margin: 0; padding: 20px; background: #1e293b; color: white; }}
        </style>
    </head>
    <body>
        <div style="background:#1e293b;padding:16px;border-radius:10px;width:800px">
            <div style="text-align:center;margin-bottom:20px;padding:15px;background:#0f172a;border-radius:8px;border:2px solid #d4af37">
                <h1 style="margin:0;font-family:'Inter',-apple-system,BlinkMacSystemFont,'Segoe UI',Roboto,Arial,sans-serif;font-size:24px;color:#d4af37;font-weight:bold">Page {page_num}</h1>
                <h2 style="margin:5px 0 0 0;font-family:'Inter',-apple-system,BlinkMacSystemFont,'Segoe UI',Roboto,Arial,sans-serif;font-size:18px;color:#e2e8f0;font-weight:normal">{surah_titles}</h2>
            </div>
            {''.join(arabic_translation)}
        </div>
    </body>
    </html>
    """
    
    # Create temp file
    with tempfile.NamedTemporaryFile(mode='w', suffix='_translation.html', delete=False, encoding='utf-8') as f:
        f.write(translation_html)
        translation_file = f.name
    
    translation_output = f"quran_page_{page_num}_arabic_translation.png"
    
    try:
        # Save Arabic + translation
        result = subprocess.run([
            sys.executable, "-c", f"""
import asyncio
from playwright.async_api import async_playwright

async def main():
    async with async_playwright() as p:
        browser = await p.chromium.launch()
        page = await browser.new_page()
        await page.goto('file://{translation_file}')
        await page.screenshot(path='{translation_output}', full_page=True)
        await browser.close()

asyncio.run(main())
"""
        ], capture_output=True, text=True)
        
        if result.returncode == 0:
            print(f"✅ Arabic + translation saved as {translation_output}")
            # Mark page as revised
            revised_pages = load_revised_pages()
            revised_pages.add(page_num)
            save_revised_pages(revised_pages)
            print(f"📝 Page {page_num} marked as revised")
            os.unlink(translation_file)
        else:
            print(f"❌ Error: {result.stderr}")
    except Exception as e:
        print(f"❌ Error: {e}")

# Get random unrevised page and save image
page_num = get_random_unrevised_page()
if page_num:
    print(f"🎲 Randomly selected page: {page_num}")
    save_page_as_image(page_num)
else:
    print("All pages have been revised!")


🎲 Randomly selected page: 600
✅ Arabic + translation saved as quran_page_600_arabic_translation.png
📝 Page 600 marked as revised
